# 1. Generate Custom Search Queries

## 1.1 Import Libraries

In [1]:
import requests, urllib.parse, json, copy, time, random

## 1.2 Create Custom SPARQL Queries

### 1.2.1 MeSH Endpoint

In [8]:
def query_descriptor_Properties(descriptorIdentifier):
    queryStr = f"""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
        PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
        
        SELECT ?label ?identifier ?treeNumLabel ?type ?annotation ?note
        FROM <http://id.nlm.nih.gov/mesh>
        WHERE {{
            mesh:{descriptorIdentifier} rdfs:label ?label .
            mesh:{descriptorIdentifier} meshv:identifier ?identifier .
            mesh:{descriptorIdentifier} meshv:treeNumber ?treeNum . 
            ?treeNum rdfs:label ?treeNumLabel .
            
            mesh:{descriptorIdentifier} rdf:type ?type .
            OPTIONAL {{ mesh:{descriptorIdentifier} meshv:annotation ?annotation . }}
            OPTIONAL {{ mesh:{descriptorIdentifier} meshv:preferredConcept ?preferredConcept . }}
            OPTIONAL {{ ?preferredConcept meshv:scopeNote ?note . }}
        }} 
    """
    return queryStr

def query_Descendants(descriptorIdentifier):
    queryStr = f"""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
        PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
        
        SELECT ?label ?identifier ?treeNumLabel ?type ?annotation ?note
        FROM <http://id.nlm.nih.gov/mesh>
        
        WHERE {{
            mesh:{descriptorIdentifier} meshv:treeNumber ?ancestorTreeNum .
            ?childTreeNum meshv:parentTreeNumber+ ?ancestorTreeNum .
            ?descriptor rdfs:label ?label .
            ?descriptor meshv:identifier ?identifier .
            ?descriptor meshv:treeNumber ?childTreeNum .
            ?childTreeNum rdfs:label ?treeNumLabel .
            OPTIONAL {{ ?descriptor rdf:type ?type . }}
            OPTIONAL {{ ?descriptor meshv:annotation ?annotation . }}
            OPTIONAL {{ ?descriptor meshv:preferredConcept ?preferredConcept . }}
            OPTIONAL {{ ?preferredConcept meshv:scopeNote ?note . }}
        }}
        ORDER BY ?label
    """
    return queryStr

### 1.2.2 WikiData Endpoint

In [ ]:
def query_Languages(descriptorIdentifier):
    queryStr = f"""
        SELECT ?item ?languageCode ?languageName ?label
        WHERE {{
            # Find the item with the given MeSH Descriptor ID
            ?item wdt:P486 "{descriptorIdentifier}".
        
            # Get labels in all available languages
            ?item rdfs:label ?label.
            BIND(LANG(?label) AS ?languageCode)
          
            # Get the full language name
            OPTIONAL {{
                ?lang wdt:P424 ?languageCode .
                ?lang rdfs:label ?languageName .
                FILTER(LANG(?languageName) = "en")
            }}
        }}
        ORDER BY ?languageCode
    """
    return queryStr

## 1.3 Create Query Response Handlers

### 1.3.1 MeSH Endpoint

In [ ]:
def extract_properties(json_data):
  """Extract properties from JSON data"""
  properties = {}

  # Check if bindings array is not empty
  if json_data["results"]["bindings"]:
    for item in json_data["results"]["bindings"]:
        
      # Dynamically extract key names & corresponding values
      for key, value in item.items():
        if key == "type":
          continue

        properties[key] = value.get("value", "")

  return properties

def extract_descendants(json_data):
  """Extracts descendants from JSON data."""
  descendants = []
  # Check if bindings array is not empty
  if json_data["results"]["bindings"]:
    for item in json_data["results"]["bindings"]:
      descendant = {}
      # Dynamically extract key names & corresponding values
      for key, value in item.items():
        if key == "type":
          continue
        descendant[key] = value.get("value", "")
      descendants.append(descendant)

  return descendants

def extract_languages(json_data):
  """Extracts languages from JSON data."""
  languages = []
  # Check if bindings array is not empty
  if json_data["results"]["bindings"]:
    for item in json_data["results"]["bindings"]:
      language = {}
      # Dynamically extract key names & corresponding values
      for key, value in item.items():
        if key == "type":
          continue
        language[key] = value.get("value", "")
      languages.append(language)

  return languages

### 1.3.2 WikiData Endpoint

In [ ]:
def process_languages(data, desired_language_codes):
    # Ensure there is data to process
    if not data:
        return {}

    # Assuming all entries have the same item, extract the item identifier from the first entry
    item_identifier = data[0].get('item', 'unknown_item')

    # Filter out entries with hyphenated languageCode and those not in the desired language codes list
    filtered_data = filter(lambda x: '-' not in x['languageCode'] and x['languageCode'] in desired_language_codes, data)

    # Group by languageCode
    grouped_languages = {}
    for entry in filtered_data:
        code = entry['languageCode']
        if code not in grouped_languages:
            grouped_languages[code] = []
        grouped_languages[code].append(entry)

    # Restructure data to the desired format
    processed_languages = {}
    for code, entries in grouped_languages.items():
        shortest_entry = min(entries, key=lambda x: len(x['languageName']))
        processed_languages[code] = {
            'languageName': shortest_entry['languageName'],
            'label': shortest_entry['label']
        }

    # Wrap processed languages under the common item identifier
    return {item_identifier: processed_languages}

## 1.4 Create Other Supporting Functions

### 1.4.1 HTTP Requests

In [ ]:
def send_sparql_request(base_url , query):
    """Send a SPARQL request and return the response or error message"""
    encoded_query = urllib.parse.quote(query)
    full_url = f"{base_url}?query={encoded_query}&format=JSON"

    try:
        response = requests.get(full_url)
        # Check if request was successful
        if response.status_code == 200:
            return response.json()
        else:
            # Handle non-200 status codes
            print(f"Error: {response.status_code}")
            return None
    except requests.RequestException as e:
        # Handle exceptions related to the request
        print(f"Request Error: {e}")
        return None

In [ ]:
# Calculate Requests that can be made in day due to API Limits
def requests_calculator(API_Keys_available, Num_URLs_per_category, total_classes = 237):
    # Total requests available ( 1 API key can make 100 requests daily) 
    total_requests = API_Keys_available * 100
    
    # Total URLs required
    total_URLs_required = total_classes * Num_URLs_per_category
    
    # Total URLs retrievable ( 1 request can get 10 URLs) 
    total_URLs_retrievable = total_requests * 10
    
    start_index = 0

    if total_URLs_retrievable >= total_URLs_required:
        # Enough requests to cover all URLs
        end_index = total_classes - 1
        print(f"Can get: {total_classes} classes")
        print("Cannot get: 0 classes")
        print(f"Start index: {start_index}, End index: {end_index}")
    else:
        # Not enough requests to cover all URLs
        retrievable_classes = total_URLs_retrievable // Num_URLs_per_category
        end_index = retrievable_classes - 1
        difference = total_classes - retrievable_classes
        print(f"Can get: {retrievable_classes} classes")
        print(f"Cannot get: {difference} classes")
        print(f"Start index: {start_index}, End index: {end_index}")
        
requests_calculator(API_Keys_available=3, Num_URLs_per_category=10)

### 1.4.2 Export data to static JSON file

In [ ]:
def write_json_file(path, data):
    """Write data to a static JSON file"""
    with open(path, 'w') as json_file:
        json.dump(data, json_file, indent=4)
        print(f"Data successfully written to {path}")

### 1.4.3 Build Tree Structure

#### Labels as Index

In [ ]:
# Labels as Index
class TreeNode:
    # Constructor for TreeNode class
    def __init__(self, **attributes):
        ''' 
        Initialize a new TreeNode with given attributes and an empty list of children
        **attributes allows passing a variable number of keyword arguments
        '''
        # Store attributes of the node (eg. label, identifier)
        self.attributes = attributes
        # Initialize an empty list to hold child nodes
        self.children = []

    # Add a child node to this TreeNode
    def add_child(self, child):
        # 'child' is expected to be an instance of TreeNode
        # Append the given child node to the list of children
        self.children.append(child)

    # Convert TreeNode(and its children recursively) into a dictionary
    def to_dict(self):
        '''
         This is useful for JSON serialization
        '''
        return {
            # Unpack and include all attributes of the node
            **self.attributes,
            # Recursively convert all children to dictionaries
            'children': [child.to_dict() for child in self.children]
        }

#### Build Tree Recursively

In [10]:
def build_tree(nodes, root_properties):
    # Initialize root node with the given root properties
    root = TreeNode(**root_properties)
    parent_tree_num = root_properties.get('treeNumLabel', '')
    
    # Iterate through all nodes found in 'nodes' dictionary
    for tree_num, node in nodes.items():
        if tree_num.startswith(parent_tree_num) and tree_num.count('.') == parent_tree_num.count('.') + 1:
            # Create a new TreeNode for the child node with its attributes
            child_node = TreeNode(**node)
            root.add_child(child_node)
            # Recursively build the tree for this child node, treating it as a new root for its sub-tree
            build_tree_recursive(nodes, child_node, tree_num)

    # Return a fully constructed tree with root node at the top
    return root

def build_tree_recursive(nodes, parent_node, parent_tree_num):
    for tree_num, node in nodes.items():
        if tree_num.startswith(parent_tree_num) and tree_num.count('.') == parent_tree_num.count('.') + 1:
            child_node = TreeNode(**node)
            parent_node.add_child(child_node)
            build_tree_recursive(nodes, child_node, tree_num)

## 1.5 Execute SPARQL Queries

### 1.5.1 MeSH Endpoint

In [7]:
mesh_url = "http://id.nlm.nih.gov/mesh/sparql"
root_identifier = 'D012871'
root_descendants = extract_descendants(send_sparql_request(mesh_url,query_Descendants(root_identifier)))
root_properties = extract_properties(send_sparql_request(mesh_url,query_descriptor_Properties(root_identifier)))
print(root_properties) 

{'label': 'Skin Diseases', 'identifier': 'D012871', 'treeNumLabel': 'C17.800', 'annotation': 'general; prefer specifics; inflammatory disease = DERMATITIS & its specifics; dryness of skin goes here: do not interpret as XERODERMA see ICHTHYOSIS unless indicated by text', 'note': 'Diseases involving the DERMIS or EPIDERMIS.'}


In [8]:
write_json_file('../02. Datasets/root_descendants.json', root_descendants)

Data successfully written to ../02. Datasets/root_descendants.json


In [9]:
def remove_node_attribute(node, attribute):
    if attribute in node:
        del node[attribute]
    for child in node.get('children', {}).values():
        remove_node_attribute(child, attribute)

try:
    data = copy.deepcopy(root_descendants)

    # Preprocess data into a dictionary with 'treeNumLabel' as keys
    nodes = {item['treeNumLabel']: item for item in data}

    # Build the tree starting from root node
    tree_structure = build_tree(nodes, root_properties)

    # Convert to dict & write to json file
    tree_dict = tree_structure.to_dict()

    # Get root label
    root_label = root_properties.get('label', 'root')

    # Use root label as the parent key name of root node
    tree_dict = {root_label: tree_dict}

    # tree_dict[root_label] to get the actual root node dictionary
    remove_node_attribute(tree_dict[root_label], 'label')
    
    # Write to json file
    write_json_file('../02. Datasets/mesh_data.json', tree_dict)

    # Convert to json and print
    json_tree = json.dumps(tree_dict, indent=4)
    print(json_tree)
    
except Exception as e:
    print(f"An error occurred: {e}")
    if 'tree_dict' in locals():
        print("Writing partially processed data to JSON file...")
        write_json_file('../02. Datasets/mesh_data_partial.json', tree_dict)

Data successfully written to ../02. Datasets/mesh_data.json
{
    "Skin Diseases": {
        "identifier": "D012871",
        "treeNumLabel": "C17.800",
        "annotation": "general; prefer specifics; inflammatory disease = DERMATITIS & its specifics; dryness of skin goes here: do not interpret as XERODERMA see ICHTHYOSIS unless indicated by text",
        "note": "Diseases involving the DERMIS or EPIDERMIS.",
        "children": {
            "Acneiform Eruptions": {
                "identifier": "D017486",
                "treeNumLabel": "C17.800.030",
                "annotation": "general or unspecified; prefer specifics",
                "note": "Visible efflorescent lesions of the skin caused by acne or resembling acne. (Dorland, 28th ed, p18, 575)",
                "children": {
                    "Acne Keloid": {
                        "identifier": "D000153",
                        "treeNumLabel": "C17.800.030.030",
                        "annotation": "acneiform eruptio

### 1.5.2 WikiData Endpoint

In [ ]:
def get_Languages(identifier):
    languages = send_sparql_request(wikidata_url, query_Languages(identifier))
    languages = extract_languages(languages)
    languages = process_languages(languages,retain_languages)
    return languages

In [ ]:
wikidata_url = "https://query.wikidata.org/sparql"
# Arabic, German, English, Spanish, French, Hindi, Japanese, Korean, Malay, Dutch, Portuguese, Russian, Tamil, Vietnamese, Chinese
retain_languages = ['ar', 'de', 'en', 'es', 'fr', 'hi', 'ja', 'ko', 'ms', 'nl', 'pt', 'ru', 'ta', 'vi', 'zh']

Generate languages for each tree node

In [ ]:
# Pre-order Traversal
def traverse_and_merge(node):
    if "identifier" in node:
        identifier = node["identifier"]
        print(f"Visiting node with identifier: {identifier}")
        # Overcome rate limit: Add a random pause between 2-4 seconds
        time.sleep(random.uniform(2, 4))
        languages = get_Languages(identifier)
        node["altLabels"] = languages
    for child in node.get("children", {}).values():
        traverse_and_merge(child)

with open('../02. Datasets/mesh_data.json', 'r', encoding = 'utf-8') as inFile:
    mesh_data = json.load(inFile)

traverse_and_merge(mesh_data['Skin Diseases'])
write_json_file('../02. Datasets/tree_structure.json', mesh_data)

### 1.5.3 Flatten Tree Structure

In [2]:
# Function to flatten the nested dictionary
def flatten_tree(tree):
    flat_list = []
    for key, value in tree.items():
        # Create a new dictionary with all attributes of the current item
        item_info = {'label':key, **value}  # Unpack all key-value pairs from value

        # Exclude the 'children' attribute from the item_info as it will be processed separately
        item_info.pop('children', None)

        # Add the current item to the list
        flat_list.append(item_info)

        # If the item has children, recursively flatten them
        children = value.get("children", {})
        if children:
            flat_list.extend(flatten_tree(children))

    return flat_list

# Flatten the given JSON structure
with open('../02. Datasets/tree_structure.json', 'r', encoding = 'utf-8') as inFile:
    tree_data = json.load(inFile)
    flattened_tree = flatten_tree(tree_data)
    write_json_file('../02. Datasets/flattened_tree.json', flattened_tree)

Data successfully written to ../02. Datasets/flattened_tree.json


### 1.5.4 Create Search List File

This function matches items in flattened_list <-> leaves. traverse flattened_list, if the matching label cannot be found in sorted_leaves key, it will not be added to matched dictionary. If an item matches the keys in sorted_leaves, then the value of the corresponding key in sorted_leaves will be added as an attribute to the flattened_list item

In [ ]:
def Create_SearchList(flattened_tree, leaves):
    matched = {}
    for item in flattened_tree:
        label = item['label']
        # Check if the label is in leaves and the item is not already added
        if label in leaves and label not in matched:
            item['treeNumLabel'] = leaves[label]
            (key, value), = item['altLabels'].items()
            item['lang']= item['altLabels'][key]
            item['WikiData_identifier']= key
            item.pop('altLabels', None)
            matched[label] = item

    # Convert the dictionary back to a list
    search_list = list(matched.values())

    return search_list

write_json_file('../02. Datasets/Dataset_URLs.json', Create_SearchList(flattened_tree, leaves))

## 1.6 Data

### 1.6.1 Visualize Tree Nodes

In [2]:
# Pre-order Traversal
def traverse_tree(node, key=None, depth=0):
    # Number of tabs to apply for each depth level
    padding = "\t" * depth
    if "identifier" in node:
        # Key of current node is passed as an argument in recursive calls
        key = key
        identifier = node["identifier"]
        treeNumLabel = node["treeNumLabel"]
        print(f"{padding}{key}")
    for child_key, child in node.get("children", {}).items():
        # Pass the child's key & increase depth
        traverse_tree(child, child_key, depth + 1)
        
# Load JSON data
with open('../02. Datasets/tree_structure.json', 'r', encoding = 'utf-8') as inFile:
    tree_data = json.load(inFile)
    traverse_tree(tree_data['Skin Diseases'],key='Skin Diseases')

Skin Diseases
	Acneiform Eruptions
		Acne Keloid
		Acne Vulgaris
			Acne Conglobata
		Chloracne
	Angiolymphoid Hyperplasia with Eosinophilia
	Breast Diseases
		Breast Cyst
		Breast Neoplasms
			Breast Carcinoma In Situ
			Breast Neoplasms, Male
			Carcinoma, Ductal, Breast
			Carcinoma, Lobular
			Hereditary Breast and Ovarian Cancer Syndrome
			Inflammatory Breast Neoplasms
			Triple Negative Breast Neoplasms
			Unilateral Breast Neoplasms
		Fibrocystic Breast Disease
		Gynecomastia
		Lactation Disorders
			Galactorrhea
				Chiari-Frommel Syndrome
		Mastitis
			Granulomatous Mastitis
	Calcinosis Cutis
	Cutaneous Fistula
	Dermatitis
		Acrodermatitis
		Dandruff
		Dermatitis Herpetiformis
		Dermatitis, Atopic
		Dermatitis, Contact
			Dermatitis, Allergic Contact
				Dermatitis, Photoallergic
				Dermatitis, Toxicodendron
			Dermatitis, Irritant
				Dermatitis, Phototoxic
				Diaper Rash
			Dermatitis, Occupational
		Dermatitis, Exfoliative
		Dermatitis, Perioral
		Dermatitis, Seborrheic


### 1.6.2 Get Leaves from Tree Structure

In [3]:
Leaves = {}
def traverse_tree(node, key=None, depth=0):
     # Check if current node is a leaf node(no children)
    is_leaf = not node.get("children")
    if is_leaf:
        # If key already exist in Leaves, append node's treeNumLabel to the key's existing list(array)
        # This will group keys to ensure no duplicated leaf names
        if key in Leaves:
            Leaves[key].append(node['treeNumLabel'])
        # If key doesn't exist in Leaves, create a new key-value pair entry in Leaves
        else:
            Leaves[key] = [node['treeNumLabel']]
    for child_key, child in node.get("children", {}).items():
        traverse_tree(child, child_key, depth + 1)

# Load JSON data
with open('../02. Datasets/tree_structure.json', 'r', encoding='utf-8') as inFile:
    tree_data = json.load(inFile)
    traverse_tree(tree_data['Skin Diseases'], key='Skin Diseases')
    # Sort leaves by key & treeNumLabels by value
    leaves = {k: sorted(v) for k, v in sorted(Leaves.items())}
    print(leaves)

{'Acantholysis': ['C17.800.865.070'], 'Acanthosis Nigricans': ['C17.800.621.430.530.100'], 'Acne Conglobata': ['C17.800.030.150.500', 'C17.800.794.111.500'], 'Acne Keloid': ['C17.800.030.030', 'C17.800.329.500.261'], 'Acrodermatitis': ['C17.800.174.100', 'C17.800.804.066'], 'Actinomycosis, Cervicofacial': ['C17.800.838.765.110'], 'Acute Generalized Exanthematous Pustulosis': ['C17.800.174.600.174'], 'Adiposis Dolorosa': ['C17.800.463.249'], 'Albinism, Ocular': ['C17.800.621.440.102.090', 'C17.800.827.080.090'], 'Alopecia Areata': ['C17.800.329.937.122.147'], 'Alternariosis': ['C17.800.838.208.416.125'], 'Anetoderma': ['C17.800.804.108'], 'Angiolymphoid Hyperplasia with Eosinophilia': ['C17.800.060'], 'Angiomatosis, Bacillary': ['C17.800.838.765.150', 'C17.800.862.060'], 'Argyria': ['C17.800.621.166'], 'Arthritis, Psoriatic': ['C17.800.859.675.175'], 'Behcet Syndrome': ['C17.800.827.368.250', 'C17.800.862.150'], 'Blastomycosis': ['C17.800.838.208.055'], 'Blister': ['C17.800.865.187'], '

In [4]:
len(leaves)

237

### 1.6.3 Get Node Path (backtrack)

In [2]:
def get_Node_Path(search_list, treeNumLabel):
    path = []
    
    # Split treeNumLabel into it's hierarchical levels
    level = treeNumLabel.split('.')

    # Iterate over each level of the treeNumLabel
    for i in range(1, len(level) + 1):
        # Construct a partial_label
        partial_label = '.'.join(level[:i])

        # Search for a node that matches the partial label
        node = next((item for item in search_list if item["treeNumLabel"] == partial_label), None)

        # If a node is found, add it to the path
        if node is not None:
            path.append(node)

    return path

def structure_path(path):
    structured_dict = {}

    # Recursively build the nested dictionary
    def build_structure(current_dict, nodes):
        if nodes:
            node = nodes[0]
            # Use label as key & create a dictionary for each node
            current_dict[node['label']] = {
                'identifier': node['identifier'],
                'treeNumLabel': node['treeNumLabel'],
                'note': node.get('note', ''),
                'children': {}
            }
            # Recurse with the next node and the 'children' dictionary of the current node
            build_structure(current_dict[node['label']]['children'], nodes[1:])

    build_structure(structured_dict, path)
    return structured_dict

with open('../02. Datasets/Dataset_URLs.json', 'r', encoding = 'utf-8') as inFile:
    search_list = json.load(inFile)
    
treeNumLabel_to_find = "C17.800.827.250.856.500"
path_to_node = get_Node_Path(search_list, treeNumLabel_to_find)
structured_path = structure_path(path_to_node)
print(structured_path)

{'Skin Diseases': {'identifier': 'D012871', 'treeNumLabel': 'C17.800', 'note': 'Diseases involving the DERMIS or EPIDERMIS.', 'children': {'Skin Diseases, Genetic': {'identifier': 'D012873', 'treeNumLabel': 'C17.800.827', 'note': 'Diseases of the skin with a genetic component, usually the result of various inborn errors of metabolism.', 'children': {'Ectodermal Dysplasia': {'identifier': 'D004476', 'treeNumLabel': 'C17.800.827.250', 'note': 'A group of hereditary disorders involving tissues and structures derived from the embryonic ectoderm. They are characterized by the presence of abnormalities at birth and involvement of both the epidermis and skin appendages. They are generally nonprogressive and diffuse. Various forms exist, including anhidrotic and hidrotic dysplasias, FOCAL DERMAL HYPOPLASIA, and aplasia cutis congenita.', 'children': {'Pachyonychia Congenita': {'identifier': 'D053549', 'treeNumLabel': 'C17.800.827.250.856', 'note': 'A group of inherited ectodermal dysplasias wh

In [7]:
def generate_node_filePath(structured_dict, path=[], paths=None):
    # Initialize paths as an empty list if not provided
    if paths is None:
        paths = []

    for key, value in structured_dict.items():
        # Append current key to the path
        new_path = path + [key]

        # If there are children, continue traversing
        if value['children']:
            generate_node_filePath(value['children'], new_path, paths)
        # If a leaf node is reached, append each item in the path separately to paths
        else:
            # Append each element of new_path to paths
            paths.extend(new_path)

    return paths
    
a= generate_node_filePath(structured_path)
print(a)
print('/'.join(a))

['Skin Diseases', 'Skin Diseases, Genetic', 'Ectodermal Dysplasia', 'Pachyonychia Congenita', 'Steatocystoma Multiplex']
Skin Diseases/Skin Diseases, Genetic/Ectodermal Dysplasia/Pachyonychia Congenita/Steatocystoma Multiplex


In [8]:
print('/'.join(generate_node_filePath(structure_path(get_Node_Path(search_list, "C17.800.030.030")))))
print('/'.join(generate_node_filePath(structure_path(get_Node_Path(search_list, "C17.800.329.500.261")))))

Skin Diseases/Acneiform Eruptions/Acne Keloid
Skin Diseases/Hair Diseases/Folliculitis/Acne Keloid
